In [1]:
import os
import pandas as pd
import numpy as np

### 원본 학습 데이터 불러오기

In [2]:
data_dir = "../data"
preprocessed_output_dir = "../data/train" # where to be saved
standard = "../data/standard" # where to be loaded

In [5]:
data_df = pd.read_csv(os.path.join(standard, "train_ratebeer_without_text.csv"))

### 2. 리뷰 개수가 1개인 유저 제거
- 리뷰 수가 1개인 유저는 모델이 학습하기에 적합하지 않은 데이터이기 때문에, 제외

In [6]:
tobe_deleted_users = (data_df.user_id.value_counts() == 1).where(lambda x: x==True).dropna().index.tolist()

In [7]:
tobe_deleted_users
train_df = data_df[~data_df["user_id"].isin(tobe_deleted_users)]
train_df.index = range(len(train_df))
train_df.head(1)
print(len(train_df))

2906971


In [8]:
# attribute extraction
style_df = train_df.groupby("beer_id")["beer_id", "style"].head(1)
style_df.index = range(len(style_df))
style_df.head(5)

<ipython-input-8-ac2becad0610>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  style_df = train_df.groupby("beer_id")["beer_id", "style"].head(1)


,beer_id,style
0,131594,Saison / Farmhouse / Grisette
1,121536,Specialty Grain - Other
2,12617,Pale Lager - American
3,14940,IPA - English
4,25933,Blonde Ale / Golden Ale


In [9]:
array, index = pd.factorize(style_df["style"])
style_df["style"] = array
style_df.groupby("beer_id")["style"].apply(list).to_json(os.path.join(preprocessed_output_dir, "rb_item2attributes.json"))

### 데이터 전처리
- 모델 학습에 필요한 데이터를 제외

In [10]:
training_data = train_df[["user_id", "beer_id", "review_score", "review_time"]]
training_data.columns = ["user", "item", "rating", "time"]
training_data = training_data.sort_values(["user", "time"])
training_data.index = range(len(training_data))
training_data.isna().sum()

user      0
item      0
rating    0
time      0
dtype: int64

In [11]:
training_data.to_csv(os.path.join(preprocessed_output_dir, "train_ratings.csv"), index=False)